In [ ]:
print("Ok")

In [ ]:
%pwd

In [3]:
import os
os.chdir("../")

In [ ]:
%pwd

In [5]:
from langchain.document_loaders import PyPDFLoader, DirectoryLoader
from langchain.text_splitter import RecursiveCharacterTextSplitter

In [6]:
#Extract Data From the PDF File
def load_pdf_file(data):
    loader= DirectoryLoader(data,
                            glob="*.pdf",
                            loader_cls=PyPDFLoader)

    documents=loader.load()

    return documents


In [7]:
extracted_data=load_pdf_file(data='Data/')

In [ ]:
extracted_data

In [10]:
#Split the Data into Text Chunks
def text_split(extracted_data):
    text_splitter=RecursiveCharacterTextSplitter(chunk_size=500, chunk_overlap=20)
    text_chunks=text_splitter.split_documents(extracted_data)
    return text_chunks

In [ ]:
text_chunks=text_split(extracted_data)
print("Length of Text Chunks", len(text_chunks))

In [12]:
# text_chunks

In [ ]:
!pip install langchain_google_genai

In [14]:
# from langchain.embeddings import HuggingFaceEmbeddings
from langchain_google_genai import GoogleGenerativeAIEmbeddings

In [15]:
#Download the Embeddings from Hugging Face
# def download_hugging_face_embeddings():
#     embeddings=HuggingFaceEmbeddings(model_name='sentence-transformers/all-MiniLM-L6-v2')
#     return embeddings
import os
from dotenv import load_dotenv

load_dotenv()

GOOGLE_API_KEY = os.environ.get("GOOGLE_API_KEY")

def get_google_embeddings():
    embeddings = GoogleGenerativeAIEmbeddings(
        model="models/embedding-001",  # Google's text embedding model
        google_api_key=GOOGLE_API_KEY
    )
    return embeddings

In [16]:
# embeddings = download_hugging_face_embeddings()

embeddings = get_google_embeddings()

In [ ]:
query_result = embeddings.embed_query("Hello world")
print("Length", len(query_result))

In [18]:
# query_result

In [ ]:
from dotenv import load_dotenv
load_dotenv()

In [20]:
PINECONE_API_KEY=os.environ.get('PINECONE_API_KEY')
# OPENAI_API_KEY=os.environ.get('OPENAI_API_KEY')
GOOGLE_API_KEY=os.environ.get('GOOGLE_API_KEY')

In [ ]:
!pip install pinecone

In [ ]:
# !pip uninstall pinecone-plugin-inference
!pip install --upgrade pinecone-client

In [ ]:
!pip install --upgrade pinecone

In [ ]:
from pinecone.grpc import PineconeGRPC as Pinecone
from pinecone import ServerlessSpec
import os

pc = Pinecone(api_key=PINECONE_API_KEY)

index_name = "medicalbot"


pc.create_index(
    name=index_name,
    dimension=768, 
    metric="cosine", 
    spec=ServerlessSpec(
        cloud="aws", 
        region="us-east-1"
    ) 
) 

# import pinecone
# from pinecone import ServerlessSpec
# # Initialize Pinecone
# pinecone.init(api_key=PINECONE_API_KEY, environment="us-east-1")

# index_name = "medicalbot"

# # Create an index
# pinecone.create_index(
#     name=index_name,
#     dimension=768,
#     metric="cosine",
#     spec=ServerlessSpec(cloud="aws", region="us-east-1")
# )



In [23]:
import os
os.environ["PINECONE_API_KEY"] = PINECONE_API_KEY
# os.environ["OPENAI_API_KEY"] = OPENAI_API_KEY

os.environ["GOOGLE_API_KEY"] = GOOGLE_API_KEY

In [ ]:
# !pip install langchain_pinecone

In [ ]:
# Embed each chunk and upsert the embeddings into your Pinecone index.
from langchain_pinecone import PineconeVectorStore

docsearch = PineconeVectorStore.from_documents(
    documents=text_chunks,
    index_name=index_name,
    embedding=embeddings, 
)

In [35]:
# Load Existing index 

from langchain_pinecone import PineconeVectorStore
# Embed each chunk and upsert the embeddings into your Pinecone index.
docsearch = PineconeVectorStore.from_existing_index(
    index_name=index_name,
    embedding=embeddings
)

In [ ]:
docsearch

In [37]:
retriever = docsearch.as_retriever(search_type="similarity", search_kwargs={"k":3})

In [38]:
retrieved_docs = retriever.invoke("What is Acne?")

In [ ]:
retrieved_docs

In [40]:
# from langchain_openai import OpenAI
# llm = OpenAI(temperature=0.4, max_tokens=500)

from langchain_google_genai import ChatGoogleGenerativeAI

llm = ChatGoogleGenerativeAI(model="gemini-1.5-pro",temperature=0.3, max_tokens=500)

In [41]:
from langchain.chains import create_retrieval_chain
from langchain.chains.combine_documents import create_stuff_documents_chain
from langchain_core.prompts import ChatPromptTemplate


system_prompt = (
    "You are an assistant for question-answering tasks. "
    "Use the following pieces of retrieved context to answer "
    "the question. If you don't know the answer, say that you "
    "don't know. Use three sentences maximum and keep the "
    "answer concise."
    "\n\n"
    "{context}"
)


prompt = ChatPromptTemplate.from_messages(
    [
        ("system", system_prompt),
        ("human", "{input}"),
    ]
)

In [42]:
question_answer_chain = create_stuff_documents_chain(llm, prompt)
rag_chain = create_retrieval_chain(retriever, question_answer_chain)

In [ ]:
response = rag_chain.invoke({"input": "what is Acromegaly and gigantism?"})
print(response["answer"])

In [ ]:
response = rag_chain.invoke({"input": "What is stats?"})
print(response["answer"])